## Load Data

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

data_path = Path('../../data/f1_fan_track')
data_files = list(data_path.glob('*.csv'))
data_files

[PosixPath('../../data/f1_fan_track/driver_pts.csv'),
 PosixPath('../../data/f1_fan_track/team_price.csv'),
 PosixPath('../../data/f1_fan_track/team_pts.csv'),
 PosixPath('../../data/f1_fan_track/driver_price.csv')]

In [2]:
frames = {}

for f in data_files:
    frames[f.stem] = pd.read_csv(f, sep='|')


In [3]:
pts_driver = frames.get('driver_pts').copy()
pts_team = frames.get('team_pts').copy()
price_driver = frames.get('driver_price').copy()
price_team = frames.get('team_price').copy()

## Clean and Calculate

### Points Tables

In [4]:
track_cols = [col for col in pts_driver.columns if (len(col) == 3 and col.isupper()) or col.endswith('.1') or col.endswith('.2')]
track_cols

['BAH',
 'SAU',
 'AUS',
 'ITA',
 'USA',
 'SPA',
 'MON',
 'AZE',
 'CAN',
 'BRI',
 'AUS.1',
 'FRA',
 'HUN',
 'BEL',
 'NET',
 'ITA.1',
 'SIN',
 'JAP',
 'USA.1',
 'MEX',
 'BRA',
 'ABU']

#### Tracks with Scores

In [6]:
keep_track_cols = pts_driver[track_cols].sum()[pts_driver[track_cols].sum() > 0].index

#### Points Metrics

In [9]:
def calc_metrics(df):

    df = df.replace(0, np.nan).dropna(axis=1)

    df['avg'] = df[keep_track_cols].mean(axis=1)
    df['max'] = df[keep_track_cols].max(axis=1)
    df['median'] = df[keep_track_cols].median(axis=1)
    
    return df

pts_driver = calc_metrics(pts_driver)
pts_team = calc_metrics(pts_team)


driver_total_avg_points = pts_driver['avg'].sum()  # The total points on average for all drivers
car_total_avg_points = pts_team['avg'].sum()

### Price Tables

In [10]:
def fix_prices(df):
    for col in ['Current Price', 'Season Start PriceSeason Price']: 
        df[col] = df[col].str.replace('$', '').str.replace('m', '').astype(float)

    return df

price_driver = fix_prices(price_driver)
price_team = fix_prices(price_team)

/var/folders/1t/72nlh0w15gzfhdb9xjzgrm1h0000gn/T/ipykernel_54831/3083300969.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('$', '').str.replace('m', '').astype(float)


In [12]:
driver_total_price = price_driver['Current Price'].sum()
car_total_price = price_team['Current Price'].sum()

In [13]:
car_total_price

165.79999999999998

In [15]:
price_team['Season Start PriceSeason Price'].sum()

167.5

In [16]:
driver_total_price / (driver_total_price + car_total_price)

0.6274994383284657

In [17]:
driver_total_price / car_total_price

1.6845597104945724

## Merge